# 最优化理论

很多机器学习算法最后都归结为求解最优化问题。最优化问题一般都是求解极大值或者极小值，对于连续可微函数，在数学上有明确的[求解方法](https://blog.csdn.net/lanchunhui/article/details/51371444)。**实际问题中，函数本身可能过于复杂，比如包含指数函数、对数函数以及根号等，使得求解偏导数等于0的方程组不是那么容易，所以我们只能通过迭代的方法求近似解，称为数值计算。**

迭代法从一个初始点 $X_{0}$ 开始，通过某个映射 h 从 $X_{k}$ 移动到下一个点 $X_{k+1}$ ，构造数列 $\{X_k\}_{k\in (1,n)}$，直到收敛到梯度为0的点处。即有下面的极限成立：

$$ \lim_{k \rightarrow + \infty} \nabla f(x_k) = 0$$


一般会利用一阶导数信息即梯度或者二阶导数信息即Hessian矩阵。迭代法的核心是得到迭代公式：

$$x_{k+1} = h(x_k)$$


## 梯度下降法

### 梯度下降法原理

**梯度下降法是通过沿着梯度下降（n=1对应的就是导数）的方向来求解近似极小值。**

我们下面针对一元**连续可微凸函数（也是应用梯度下降法的前提，分段连续可微是否可行呢？理论上也是可行的，分段求解即可）**（即必然有最小值）来推导梯度下降法的一般公式（一种解释[链接](http://www.cnblogs.com/ljygoodgoodstudydaydayup/p/7274943.html)，多元情形请参考[链接](https://zhuanlan.zhihu.com/p/36902908)）。一元函数的一阶泰勒公式展开为：

$$f(x + \Delta x) = f(x) + f^\prime(x)\Delta x + o(\Delta x)$$

即 
$$f(x + \Delta x) - f(x) = f^\prime(x)\Delta x + o(\Delta x)$$

为了保证每次迭代使得 $f(x + \Delta x) - f(x) < 0$ ,要有 $ f^\prime(x)\Delta x < 0 $ 。因为凸函数的梯度 $f^\prime(x) > 0 $，所以要保证 $\Delta x < 0$。

那么每次 $\Delta f = f(x + \Delta x) - f(x)$ 下降多少取决于 $|f^\prime(x)\Delta x|$ 的大小。对于一个固定的点 $ f^\prime(x) $ 是常数，所以 $\Delta f$ 的大小由 $\Delta x $ 决定。不妨设

$$ \Delta x = - \alpha f^\prime(x),\quad \alpha \in (0, 1)$$

则函数下降有：

$$ f(x + \Delta x) - f(x) = - \alpha f^\prime(x)^2$$

下降速度取决于每点的梯度大小以及学习率。

$ \Delta x = - \alpha f^\prime(x)$ 假设的合理性在于：
1. 保证了 $\Delta x $ 和 $ f^\prime(x) $方向相反，_**沿着梯度相反方向下降是最快的方向**_（多元情形下，两者都是向量，夹角是$\cos \theta$）；
2. $\alpha$ 作为学习率控制了下降的幅度，取值在0到1之间，是因为如果步长太大，会越过极小值。

下面是我自己画的草图，注意到 $f^\prime(x) = \tan \theta$ ， 考虑到 

$$ \tan \theta \approx \theta \quad \theta \rightarrow 0$$

当靠近极小值的时候，有$ \Delta x = - \alpha \theta$， 最终会收敛到极小值。

最终，求解极小值点的问题变成了按照图中斜率导数所示的“梯子”方向逐步下降直到最低点的问题。

<img src="figure/梯度下降.png" width="400" hegiht="300" align=left />

实际求解中会遇到局部极小值和鞍点问题，相应的解决办法我们稍后再谈。

### 批量梯度下降算法（Batch Gradient Descent，简称BGD）

梯度下降法的原理可以看上面最优化理论的“梯度下降法原理”。

根据成本函数，梯度下降公式是：

$$\begin{split}\theta_{j+1} = &\theta_j - \alpha_{j+1} \frac{\partial J(\theta)}{\partial \theta}|_{\theta = \theta_j}\\
= &\theta_j - \frac{\alpha_{j+1}}{m}\sum_{i=1}^{m}(h(\textbf{x}^{(i)})-y^{(i)})x_j^{(i)}
\end{split}$$

如果 每一步的学习率是一致不变的，可以记为 $\alpha$.

编程语言实现如下：
1. 确定学习率和初始的参数起点，学习率不能过大，否则会无法收敛；过小会导致计算太多，算法效率低；
2. 迭代：根据梯度下降参数迭代公式，算出新的参数向量，得到新的预测函数，进而得到新的成本；
3. 确认成本函数是否收敛：对比成本函数的变化，看成本是否越来越小，如果两次成本差值小于误差范围，停止迭代，否则继续。

### 随机梯度下降算法(Stochastic Gradient Descent，简称SGD)
上面的梯度下降法中每次循环更新参数需要遍历所有的训练数据集，我们可以改成每次随机从训练数据集里抽取一个数据进行迭代计算，这就是**随机梯度下降法**，公式如下：

$$\theta_j = \theta_j - \alpha(h(\textbf{x}^{(i)})-y^{(i)})x_j^{(i)}$$

**随机梯度下降算法和梯度下降算法在数学上证明是等价的。**

SGD的优缺点：
* 对于训练数据size比较大的训练集，训练速度快，大概是 O(n) 和 $O(2^n)$ 的对比；
* 伴随的问题是噪音要多，使得SGD并不是每次迭代都向着整体最优化方向。但是由于更新多轮，整体方向还是大致朝着极小值方向更新。
* SGD和MBGD相比BGD,优点是提升了效率，但算法最终不会收敛到最优点，会在最优点附近波动。

为了更加接近最优值，可以逐渐降低学习率：

**开始时，走的每一步较大（这有助于快速前进同时跳过局部最小值），然后变得越来越小，从而使算法到达全局最小值**。 这个过程被称为**模拟退火。**

### 小批量梯度下降法（Mini-batch Gradient Descent，简称MBGD）

MBGD 是 BGD 和 SGD 的折中，为了解决批梯度下降法训练速度慢，以及随机梯度下降法的准确性综合而来,每次迭代选用b个样本进行更新。公式如下：

$$\theta_j = \theta_j - \frac{\alpha}{b}\sum_{k=i}^{i+b}(h(\textbf{x}^{(i)})-y^{(i)})x_j^{(i)}, \quad i = 1, 1+b, ...,m-b, m$$

## [最大似然估计（maximum likelihood estimation，缩写为MLE）](https://zh.wikipedia.org/wiki/%E6%9C%80%E5%A4%A7%E4%BC%BC%E7%84%B6%E4%BC%B0%E8%AE%A1)

令 $P(Y=1|X=x) = P(x;\theta)$, 其中 P 是以 $\theta$ 为参数的函数。假设样本 $x_i$ 之间相互独立，则似然函数(likelihood function)为

$$L(\theta) = \prod_{i=1}^m P(Y=y_i|X=x_i) = \prod_{i=1}^m P(x_i;\theta)^{y_i}(1-P(x_i;\theta))^{(1- y_i)}$$

对 $L(\theta)$取对数，则得到**对数似然函数**（log-likelihood function）如下：

$$\ell(\theta) = \ln L(\theta) = \sum_{i=1}^{m} y_i \ln P(x_i;\theta) + \sum_{i=1}^{m} (1-y_i) \ln (1-P(x_i;\theta))$$

一般求解会对对数似然函数求关于 $\theta$ 的最小值，利于计算和求导。称

$$ \theta^{\star} = -\arg\min_{\theta}\ell(\theta)$$

为**最小对数似然估计**，相应的最大似然估计是对 $L(\theta)$ 求 关于 $\theta$ 的最大值。

## [Lagrange 对偶与KKT条件](https://wizardforcel.gitbooks.io/dm-algo-top10/content/svm-2.html)

对于 向量 $\mathbf{x}\in R^n$，考虑一般形式的不等式约束问题：

$$\left\{\begin{aligned}
\min_{\mathbf{x}} f(\mathbf{x}) &\\
s.t. h_i(\mathbf{x}) = 0 &\quad i = 1,2,...,k \\
g_i(\mathbf{x}) \leq  0 &\quad i = 1,2,...,l
\end{aligned}\right. \tag{Primal Constraints}$$

对应的 Lagrange 公式为：

$$L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta}) = f(\mathbf{x}) + \sum_{i=1}^{k}\alpha_i g_i(\mathbf{x}) + \sum_{i=1}^{l}\beta_i h_i(\mathbf{x}) \tag{Larange Model}$$

其中 $\mathbf{\alpha}, \mathbf{\beta}$ 都是 Lagrange 算子。

我们要求解的就是满足 **primal constraints (简称PC)** 的 $\min_{\mathbf{x}}f(\mathbf{x})$。

对于 $L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta})$ 我们适当地调整 $\mathbf{\alpha}, \mathbf{\beta}$ 的值，总会有：

$$ \max_{\beta,\alpha \geq 0} L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta})=
\left \{
\begin{aligned}
f(\mathbf{x}) & \quad (\mathbf{\alpha}, \mathbf{\beta}) \; satisfy \; (PC)\\
+\infty & \quad else 
\end{aligned}\right.$$

如果记 $\theta_{PC}(\mathbf{x}) = \max_{\mathbf{\beta}, \mathbf{\alpha}\geq 0} L(x, \mathbf{\alpha}, \mathbf{\beta})$，则求解满足 PC 的 $\min_{\mathbf{x}}f(\mathbf{x})$的问题等价于

$$\begin{split}
d^\ast = \min_{\mathbf{x}}f(\mathbf{x}) =& \min_{\mathbf{x}} \theta_{PC}(\mathbf{x})\\
=& \min_{\mathbf{x}} \max_{\mathbf{\beta}, \mathbf{\alpha}\geq 0} L(\mathbf{x}, \alpha, \beta)
\end{split}$$

一般有 $\max\min f \leq \min\max f$ (从 $f \leq \max f$开始推导) ,所以有：

$$ d^\ast = \min_{\mathbf{x}}f(\mathbf{x}) =\min_{\mathbf{x}} \max_{\mathbf{\beta}, \mathbf{\alpha} \geq 0} L(\mathbf{x}, \alpha, \beta)
\geq \max_{\mathbf{\beta}, \mathbf{\alpha}\geq 0}\min_{\mathbf{x}} L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta}) = p^\ast$$

到此为止，求解不等式左侧满足约束的极小值问题转换成不等式右侧求极大值的问题。求极大值是凸规划里的常见问题，对偶问题的转换都是往利于求解的方向。

**KKT条件：**

求解满足 （PC）的极小值问题，对应的 Lagrange 公式存在极小值的必要条件是满足KKT条件：

$$\left\{\begin{aligned}
\nabla_{\mathbf{x}}  L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta}) = \mathbf{0}\\
h_i(\mathbf{x}) = 0 \\
g_i(\mathbf{x}) \leq  0  \\
\alpha_i \geq 0 \\
\alpha_i g_i = 0
\end{aligned}\right. \tag{KKT}$$
